# Azure Function

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [13]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import AzureFunctionStorageQueue, AzureFunctionTool, MessageRole
from azure.identity import DefaultAzureCredential

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [14]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup Azure Function tool

In [15]:
# Retrieve the storage service endpoint from environment variables
storage_service_endpoint = os.environ["STORAGE_SERVICE_ENDPOINT"]

# Define the Azure Function tool
azure_function_tool = AzureFunctionTool(
    name="QueueFunc",  # Name of the tool
    description="Get the weather information",  # Description of the tool's purpose
    parameters={  # Define the parameters required by the tool
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "The location to get weather information for."},
        },
    },
    input_queue=AzureFunctionStorageQueue(  # Input queue configuration
        queue_name="azure-function-input",
        storage_service_endpoint=storage_service_endpoint,
    ),
    output_queue=AzureFunctionStorageQueue(  # Output queue configuration
        queue_name="azure-function-output",
        storage_service_endpoint=storage_service_endpoint,
    ),
)

## Create the agent

In [16]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-azurefunction-agent",  # Name of the agent
    instructions="You are a helpful agent",  # Instructions for the agent
    tools=azure_function_tool.definitions
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_twjBR6ywY8lCXZ71sINCFg3N


## Create thread

In [17]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_vDMcrYKuDkUYvKmQyOpqSxQ5


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [18]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [19]:
user_input = "What is the weather in London? Provide your source."
run_agent(user_input)

Created message, ID: msg_WWnorIfolGKq0yFVkhIh6qh2
Run finished with status: completed
Agent Response: The current weather in London is cloudy with a temperature of 18°C. The information is sourced from the Azure Function.


In [20]:
user_input = "What is the weather in Paris? Provide your source."
run_agent(user_input)

Created message, ID: msg_AuUKlmAE2aDoJepzpvAIVg9R
Run finished with status: completed
Agent Response: The weather data for Paris is currently not available. The information is sourced from the Azure Function.


## Cleanup resources

In [21]:
# Cleanup resources

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted agent
